In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, ParticleFile, plotTrajectoriesFile
from parcels import AdvectionRK4
import numpy as np
from datetime import timedelta as delta
%matplotlib inline

import xarray as xr

Data needs to be downloaded to work in this notebook
Download eReefs data from https://dapds00.nci.org.au/thredds/fileServer/fx3/gbr1_2.0/gbr1_simple_2022-05-01.nc 

Have this data within the local workspace to be used.

In [ ]:
# This opens the dataset for inspection.
parcels = xr.open_dataset('gbr1_simple_2023-02-01.nc')
# dropped variables that are not required for Parcels to work
parcels = parcels.drop(['dhw0','dhw1','dhw2','dhw3','RT_expose','swr','temp_expose','salt','wspeed_u','wspeed_v'])
#drops the depth variable, so that data is smaller and easier to work with. 
parcels = parcels.sel(k=42) #represents the surface.
parcels

In [ ]:
#saves this local netcdf

#parcels.to_netcdf(path='parcels.nc')

## Drifters within eReefs

In [13]:
filenames = {'U': 'parcels.nc',
             'V': 'parcels.nc'}

variables = {'U': 'u',
             'V': 'v'}
dimensions = {'U': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'},
              'V': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'}}
field_set = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)

In [ ]:
field_set.U.show()

#This result doesn't work but doesn't impact the running of the code. 

#This is the error
'''ValueError: x and y arguments to pcolormesh cannot have non-finite values or be of type numpy.ma.core.MaskedArray with masked values'''

In [ ]:
npart = 10  # number of particles to be released
time = np.arange(0,npart)*delta(minutes=40).total_seconds()  # release every particle in a delayed mode

pset = ParticleSet.from_line(fieldset=field_set, 
                             pclass=JITParticle,
                             size=npart,             # releasing X particles
                             start=(147.6, -19.2),   # releasing on a line: the start longitude and latitude
                             finish=(147.8, -19.0),
                             time=time)  # release every particle one hour

In [ ]:
print(pset)
pset.show()

In [ ]:
output_file = pset.ParticleFile(name="CurrentParticles.zarr", outputdt=delta(hours=1))  # the file name and the time step of the outputs

pset.execute(AdvectionRK4,  # the kernel (which defines how particles move)
             runtime=delta(days=10), # the total length of the run
             dt=delta(minutes=5), # the timestep of the kernel
             output_file=output_file,
             )

output_zarr = 'CurrentParticles.zarr'
try:
    os.remove(output_zarr)
except OSError:
    pass

In [ ]:
parcels = xr.open_dataset('parcels.nc')

In [ ]:
print(pset)
pset.show()

Plotting of the files

In [ ]:
plotTrajectoriesFile(output_zarr, mode='movie2d_notebook')

In [ ]:
plotTrajectoriesFile(output_zarr, mode='hist2d', bins=[30, 20]);